In [1]:
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
from IPython.core.display import Markdown
from IPython.core.display_functions import display
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

import notebooks.config as config
import notebooks.utils as nb_utils
from src.config import FeaturesInfo
from src.features.univariate_analysis import UACleaner
from src.utils import init_cols_nan_strategy, preprocess_init

STAGE = 1
PREVIOUS_STAGE = STAGE - 1

%load_ext autoreload
%autoreload 2

## Load cleaned data and metadata in artifacts

In [2]:
df, features_info = nb_utils.load_dataset_and_metadata(
    nb_utils.STAGES_DICT[PREVIOUS_STAGE]["name"],
    nb_utils.STAGES_DICT[PREVIOUS_STAGE]["folder_path"],
)

In [3]:
df.info(memory_usage="deep", max_cols=200)
print()

max_nans_for_rows = df.isna().sum(axis=1).max()
print(f"Max NaNs for rows: {max_nans_for_rows}")
print()

pd.options.display.max_rows = None

print("NaNs for cols")
nans_for_cols = nb_utils.get_nas(df)
display(nans_for_cols[nans_for_cols > 0])

pd.reset_option("display.max_rows", 0)

<class 'pandas.core.frame.DataFrame'>
Index: 29991 entries, 9249043 to 23246511
Data columns (total 170 columns):
 #    Column                                   Non-Null Count  Dtype         
---   ------                                   --------------  -----         
 0    name                                     29991 non-null  string        
 1    short_url                                29991 non-null  string        
 2    price                                    29991 non-null  UInt32        
 3    listing_followers_no                     27205 non-null  UInt16        
 4    location                                 29991 non-null  category      
 5    images_no                                29991 non-null  UInt8         
 6    description                              29991 non-null  string        
 7    gi_brand                                 29991 non-null  category      
 8    gi_model                                 29991 non-null  category      
 9    gi_production_year    

,missing count,missing [%]
ai_range_on_full_battery_km,29864,99.576540
gi_battery_capacity,29834,99.476510
ai_interest_free_credit,29805,99.379814
ai_cash_payment,29708,99.056384
ai_deposit,29518,98.422860
ai_installment_amount,29413,98.072755
ai_installment_no,29381,97.966056
ai_leasing,29204,97.375879
gi_certified,28474,94.941816
ai_credit,28174,93.941516


In [4]:
df_train, df_test = train_test_split(
    df,
    test_size=config.TEST_SIZE,
    random_state=config.RANDOM_SEED,
)

df_orig = df
df = df_train  # Using alias for making things easier and more neat

cols_nan_strategy = init_cols_nan_strategy()
idx_to_remove = []

In [5]:
n_rows_train, n_cols_train = df_train.shape

print("Train dataset shape")
print(20 * "-")
print(f"No of rows: {n_rows_train}")
print(f"No of cols: {n_cols_train}")

print()

n_rows_test, n_cols_test = df_test.shape

print("Test dataset shape")
print(20 * "-")
print(f"No of rows: {n_rows_test}")
print(f"No of cols: {n_cols_test}")
print(20 * "-")

Train dataset shape
--------------------
No of rows: 23992
No of cols: 170

Test dataset shape
--------------------
No of rows: 5999
No of cols: 170
--------------------


### nominal features

In [7]:
@preprocess_init
def ua_nominal_features_nb(
    df: pd.DataFrame,
    features_info: FeaturesInfo,
    cols_nan_strategy: Dict[str, List[str]],
) -> Tuple[pd.DataFrame, FeaturesInfo, Dict[str, List[str]]]:
    print("Frequencies for nominal features")
    for col in features_info["nominal"]:
        df[col] = df[col].cat.remove_unused_categories()
        display(nb_utils.get_value_counts_freq_with_perc(df, col))
    print("Dropped empty categories")
    print()

    df.loc[df.gi_body_type == "", "gi_body_type"] = np.nan
    print("Replaced ' ' category for 'gi_body_type' with NaN")

    df.gi_fuel_type = pd.Categorical(
        df.gi_fuel_type.astype("string").replace(
            {
                "Hibridni pogon (Benzin)": "Hibridni pogon",
                "Hibridni pogon (Dizel)": "Hibridni pogon",
                "Plug-in hibrid": "Hibridni pogon",
                "Metan CNG": "Benzin + Metan (CNG)",
            }
        ),
        ordered=False,
    )
    print("Grouped simillar categories for 'gi_fuel_type'")

    df.ai_gearbox_type = pd.Categorical(
        df.ai_gearbox_type.astype("string").replace(
            {
                "Automatski": "Automatski / poluautomatski",
                "Poluautomatski": "Automatski / poluautomatski",
            }
        )
    )
    print("Grouped simillar categories for 'ai_gearbox_type'")

    constant_strat_cols = [
        "ai_floating_flywheel",
        "ai_interior_material",
        "ai_interior_color",
        "ai_ownership",
        "ai_import_country",
        "ai_sales_method",
    ]
    modus_strat_cols = [
        col for col in features_info["nominal"] if col not in constant_strat_cols
    ]
    cols_nan_strategy["const"].extend(constant_strat_cols)
    cols_nan_strategy["modus"].extend(modus_strat_cols)
    print("NaN constant strategy columns:")
    print(constant_strat_cols)
    print("Extended 'constant' and 'modus' column_handle_nan_dict")

    return df, features_info, cols_nan_strategy


df, features_info, cols_nan_strategy = ua_nominal_features_nb(
    df=df, features_info=features_info, cols_nan_strategy=cols_nan_strategy
)

Frequencies for nominal features


,count,percentage [%]
location,,
Beograd,3925,16.359620
Novi Sad,1682,7.010670
Niš,977,4.072191
Kragujevac,890,3.709570
Čačak,716,2.984328
...,...,...
Rudnik,1,0.004168
Rudna Glava,1,0.004168
Riđica,1,0.004168


,count,percentage [%]
gi_brand,,
Volkswagen,3855,16.067856
Audi,2503,10.432644
BMW,2173,9.057186
Opel,1886,7.860954
Peugeot,1612,6.718906
...,...,...
Great Wall,1,0.004168
Geely,1,0.004168
Cadillac,1,0.004168


,count,percentage [%]
gi_model,,
A4,677,2.821774
Octavia,499,2.079860
A3,496,2.067356
A6,480,2.000667
Golf 7,465,1.938146
...,...,...
E 230,1,0.004168
SL 280,1,0.004168
SL 300,1,0.004168


,count,percentage [%]
gi_body_type,,
Hečbek,8212,34.228076
Džip/SUV,4740,19.756586
Limuzina,4429,18.460320
Karavan,3633,15.142548
Monovolumen (MiniVan),1944,8.102701
Kupe,645,2.688396
Kabriolet/Roadster,247,1.029510
Pickup,140,0.583528
,2,0.008336


...

,count,percentage [%]
ai_vehicle_origin,,
Domaće tablice,14673,61.157886
Na ime kupca,9171,38.225242
Strane tablice,146,0.608536
<NA>,2,0.008336


,count,percentage [%]
ai_ownership,,
Vodi se na prodavca,10279,42.843448
<NA>,9338,38.921307
Ne vodi se na prodavca,4375,18.235245


,count,percentage [%]
ai_import_country,,
<NA>,17948,74.808269
Švajcarska,1609,6.706402
Italija,1109,4.622374
Francuska,995,4.147216
Nemačka,846,3.526175
Holandija,534,2.225742
Belgija,294,1.225408
Norveška,209,0.871124
Austrija,155,0.646049


,count,percentage [%]
ai_sales_method,,
<NA>,21563,89.875792
Komisiona,2210,9.211404
Konsignaciona,219,0.912804


Dropped empty categories

Replaced ' ' category for 'gi_body_type' with NaN
Grouped simillar categories for 'gi_fuel_type'
Grouped simillar categories for 'ai_gearbox_type'
NaN constant strategy columns:
['ai_floating_flywheel', 'ai_interior_material', 'ai_interior_color', 'ai_ownership', 'ai_import_country', 'ai_sales_method']
Extended 'constant' and 'modus' column_handle_nan_dict
